In [10]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm, norm
import matplotlib.pyplot as plt
import seaborn as sns

def conditional_outcome(s1, s2, result, sigma_t, e=0.1):
    mean_diff = s1 - s2
    std_diff = sigma_t

    if result == 1:
        a, b = 0, np.inf 
    elif result == -1:
        a, b = -np.inf, 0  
    else:
        a, b = -e, e  # Draw

    a_std = (a - mean_diff) / std_diff
    b_std = (b - mean_diff) / std_diff

    return a_std, b_std, mean_diff, std_diff

def gibbs_sampler_result(mu_s1, mu_s2, sigma_s1, sigma_s2, sigma_t, n_iter, result, epsilon=0.01):
    s1_samples = np.zeros(n_iter)
    s2_samples = np.zeros(n_iter)
    t_samples = np.zeros(n_iter)
    y_samples = np.zeros(n_iter)

    s1_current = mu_s1
    s2_current = mu_s2

    mu_prior = np.array([mu_s1, mu_s2]).reshape((2, 1))
    sigma_prior = np.array([[sigma_s1, 0], [0, sigma_s2]])
    sigma_prior_inv = np.linalg.inv(sigma_prior)

    A = np.array([1, -1]).reshape((1, 2))
    A_T = A.T

    for i in range(n_iter):
        a_std, b_std, mean_diff, std_diff = conditional_outcome(s1_current, s2_current, result, sigma_t, epsilon)
        t_current = truncnorm.rvs(a=a_std, b=b_std, loc=mean_diff, scale=std_diff)

        sigma_post = np.linalg.inv(sigma_prior_inv + (A_T @ A) / sigma_t**2)
        mu_post = sigma_post @ (sigma_prior_inv @ mu_prior + (A_T * t_current) / sigma_t**2)
        s1_current, s2_current = np.random.multivariate_normal(mu_post.flatten(), sigma_post)

        s1_samples[i] = s1_current
        s2_samples[i] = s2_current
        t_samples[i] = t_current
        y_samples[i] = result

    mean_s1 = np.mean(s1_samples)
    var_s1 = np.var(s1_samples, ddof=1)
    mean_s2 = np.mean(s2_samples)
    var_s2 = np.var(s2_samples, ddof=1)

    return mean_s1, var_s1, mean_s2, var_s2

def win_probability_player1(mu_s1, mu_s2, sigma_s1, sigma_s2, sigma_t):
    mu_diff = mu_s1 - mu_s2
    sigma_diff = np.sqrt(sigma_s1 + sigma_s2 + sigma_t**2)
    p_y1 = 1 - norm.cdf(0, loc=mu_diff, scale=sigma_diff)
    return p_y1

# Draw Probability
def draw_probability(mu_s1, mu_s2, sigma_s1, sigma_s2, sigma_t, epsilon=0.1):
    mu_diff = mu_s1 - mu_s2
    sigma_diff = np.sqrt(sigma_s1 + sigma_s2 + sigma_t**2)
    p_draw = norm.cdf(epsilon, loc=mu_diff, scale=sigma_diff) - norm.cdf(-epsilon, loc=mu_diff, scale=sigma_diff)
    return p_draw

data_file = 'SerieA.csv'
df = pd.read_csv(data_file)
df_filtered = df.copy()

#  1 for player 1 win, -1 for player 2 win, and 0 for draw
df_filtered['win'] = df_filtered.apply(lambda row: 1 if row['score1'] > row['score2'] 
                                        else (-1 if row['score1'] < row['score2'] else 0), axis=1)

initial_mu = 25  
initial_sigma = 8.33  
sigma_t = 25 / 6  

teams = set(df_filtered['team1']).union(set(df_filtered['team2']))
skills = {team: (initial_mu, initial_sigma) for team in teams}

n_iter = 1000  
predictions = []

for index, row in df_filtered.iterrows():
    team1 = row['team1']
    team2 = row['team2']
    result = row['win']

    mu_s1, sigma_s1 = skills[team1]
    mu_s2, sigma_s2 = skills[team2]

    mean_s1, var_s1, mean_s2, var_s2 = gibbs_sampler_result(mu_s1, mu_s2, sigma_s1, sigma_s2, sigma_t, n_iter, result)

    # Update skills
    skills[team1] = (mean_s1, var_s1)
    skills[team2] = (mean_s2, var_s2)

    
    p_y1 = win_probability_player1(mu_s1, mu_s2, sigma_s1, sigma_s2, sigma_t)
    p_draw = draw_probability(mu_s1, mu_s2, sigma_s1, sigma_s2, sigma_t)

    if p_draw > 0.5:
            predictions.append(0)  # Draw
    else:
            predictions.append(1 if p_y1 > 0.45 else -1) #set probability less so as to give the home team an advantage 

correct = 0

for i in range(len(predictions)):
    if predictions[i] == df_filtered.iloc[i]['win']:
        correct += 1

print(f"Accuracy of the model: {correct / len(predictions):.4f}")

sorted_skills = sorted(skills.items(), key=lambda x: x[1][0], reverse=True)

print("\nUpdated Skills:")
for team, (mu, sigma) in sorted_skills:
    print(f"{team}: Mean Skill = {mu:.2f}, Variance = {sigma:.2f}")


Accuracy of the model: 0.4895

Updated Skills:
Juventus: Mean Skill = 28.22, Variance = 0.73
Napoli: Mean Skill = 26.93, Variance = 1.02
Inter: Mean Skill = 26.52, Variance = 0.48
Milan: Mean Skill = 26.22, Variance = 0.79
Atalanta: Mean Skill = 26.19, Variance = 0.53
Roma: Mean Skill = 26.18, Variance = 0.59
Torino: Mean Skill = 25.85, Variance = 0.45
Lazio: Mean Skill = 25.43, Variance = 0.63
Sampdoria: Mean Skill = 24.78, Variance = 0.82
Fiorentina: Mean Skill = 24.69, Variance = 0.51
Udinese: Mean Skill = 24.52, Variance = 1.22
Bologna: Mean Skill = 24.45, Variance = 0.71
Genoa: Mean Skill = 24.37, Variance = 0.59
Sassuolo: Mean Skill = 24.35, Variance = 0.67
Parma: Mean Skill = 24.31, Variance = 0.53
Spal: Mean Skill = 24.24, Variance = 0.86
Cagliari: Mean Skill = 23.99, Variance = 0.79
Empoli: Mean Skill = 23.60, Variance = 0.67
Chievo: Mean Skill = 22.81, Variance = 0.86
Frosinone: Mean Skill = 22.69, Variance = 0.65
